# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

filepath = "../output_data/cities.csv"

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = pd.read_csv(filepath)
city_data.head()

,City,Temperature_Max,Latitude,Longitude,Humidity %,Cloudiness %,Wind Speed
0,Ushuaia,8.0,-54.80,-68.30,81,75,9.30
1,Rikitea,25.9,-23.12,-134.97,75,100,4.06
2,Gwanda,18.1,-20.93,29.00,77,90,1.10
3,Dikson,-12.6,73.51,80.55,92,0,3.74
4,Itacoatiara,23.0,-3.14,-58.44,100,71,1.04


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = city_data[['Latitude', 'Longitude']].astype(float)
humidity = city_data['Humidity %'].astype(float)

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                dissipating=False, max_intensity=100,
                                point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
idealWeather = city_data.loc[city_data['Temperature_Max']>15].loc[city_data['Cloudiness %']<25].loc[city_data['Humidity %']<25].loc[city_data['Wind Speed']<10].reset_index(drop=True)
idealWeather 

,City,Temperature_Max,Latitude,Longitude,Humidity %,Cloudiness %,Wind Speed
0,Carnarvon,37.00,-24.87,113.63,20,0,6.70
1,Portales,21.00,34.19,-103.33,18,1,5.70
2,Gouré,26.20,13.98,10.27,14,0,2.50
3,Las Cruces,23.00,32.31,-106.78,16,1,9.30
4,Umluj,20.20,25.02,37.27,21,23,1.46
5,Shirdi,29.70,19.77,74.48,24,0,4.35
6,Meadow Lake,15.56,34.80,-106.54,19,1,9.30


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = pd.DataFrame()
hotel_df['Hotel Name'] = ""

In [7]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "keyword": "hotel",
    "type": "lodging",
    "key": g_key
    }

# Use the lat/lng we recovered to identify hotels
for index, row in idealWeather.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel = requests.get(base_url, params=params)
    
    # convert to json
    hotel = hotel.json()

    try:
        hotel_df.loc[index, "Hotel Name"] = hotel['results'][0]['name']
        hotel_df.loc[index, "Lat"] = hotel['results'][0]['geometry']['location']['lat']
        hotel_df.loc[index, "Lng"] = hotel['results'][0]['geometry']['location']['lng']

       
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.


In [8]:
hotel_df = hotel_df.dropna().reset_index(drop = True)
hotel_df

,Hotel Name,Lat,Lng
0,Hospitality Carnarvon,-24.892341,113.655034
1,Holiday Inn Express & Suites Portales,34.173183,-103.357336
2,Hotel Encanto de Las Cruces,32.312685,-106.740495
3,عروس الشاطئ للوحدات السكنية المفروشة,25.040592,37.253971
4,Sun-n-Sand Shirdi,19.772953,74.479602


In [9]:
params = {
    "key": g_key
    }

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["latlng"] = f"{lat},{lng}"
    
    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    # make request and print url
    location = requests.get(base_url, params=params)
    
    # convert to json
    location = location.json()
        
    if location['results'][0]['address_components'][0]['types'] == ['locality','political']:
        hotel_df.loc[index, "City"] = location['results'][0]['address_components'][0]['long_name']
    elif location['results'][0]['address_components'][1]['types'] == ['locality','political']:
        hotel_df.loc[index, "City"] = location['results'][0]['address_components'][1]['long_name']
    elif location['results'][0]['address_components'][2]['types'] == ['locality','political']:
        hotel_df.loc[index, "City"] = location['results'][0]['address_components'][2]['long_name']
    elif location['results'][0]['address_components'][3]['types'] == ['locality','political']:
        hotel_df.loc[index, "City"] = location['results'][0]['address_components'][3]['long_name']
    elif location['results'][0]['address_components'][4]['types'] == ['locality','political']:
        hotel_df.loc[index, "City"] = location['results'][0]['address_components'][4]['long_name']
    elif location['results'][0]['address_components'][5]['types'] == ['locality','political']:
        hotel_df.loc[index, "City"] = location['results'][0]['address_components'][5]['long_name']
        
        
    if location['results'][0]['address_components'][0]['types'] == ['country','political']:
         hotel_df.loc[index, "Country"] = location['results'][0]['address_components'][0]['long_name']
    elif location['results'][0]['address_components'][1]['types'] == ['country', 'political']:
         hotel_df.loc[index, "Country"] = location['results'][0]['address_components'][1]['long_name']
    elif location['results'][0]['address_components'][2]['types'] == ['country', 'political']:
        city.loc[index, "Country"] = location['results'][0]['address_components'][2]['long_name']
    elif location['results'][0]['address_components'][3]['types'] == ['country', 'political']:
        hotel_df.loc[index, "Country"] = location['results'][0]['address_components'][3]['long_name']
    elif location['results'][0]['address_components'][4]['types'] == ['country', 'political']:
        hotel_df.loc[index, "Country"] = location['results'][0]['address_components'][4]['long_name']
    elif location['results'][0]['address_components'][5]['types'] == ['country', 'political']:
        hotel_df.loc[index, "Country"] = location['results'][0]['address_components'][5]['long_name']
    elif location['results'][0]['address_components'][6]['types'] == ['country', 'political']:
        hotel_df.loc[index, "Country"] = location['results'][0]['address_components'][6]['long_name']

In [10]:
hotel_df

,Hotel Name,Lat,Lng,City,Country
0,Hospitality Carnarvon,-24.892341,113.655034,South Carnarvon,Australia
1,Holiday Inn Express & Suites Portales,34.173183,-103.357336,Portales,United States
2,Hotel Encanto de Las Cruces,32.312685,-106.740495,Las Cruces,United States
3,عروس الشاطئ للوحدات السكنية المفروشة,25.040592,37.253971,Umluj,Saudi Arabia
4,Sun-n-Sand Shirdi,19.772953,74.479602,Shirdi,India


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers_layer)
fig

fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(markers_layer)

# Display Map
fig


Figure(layout=FigureLayout(height='420px'))